# Admin Panel

> Flexible, transparent admin panel utilities for FastHTML applications

In [ ]:
#| default_exp admin

## Imports and utils

In [ ]:
#| export
from typing import Optional, Dict, Any, List, Union, Callable, Set, Type
from dataclasses import dataclass, field, fields
from datetime import datetime, date
from fasthtml.common import *
from fastlite import *
from monsterui.all import *
from launch_kit.permissions import require_role
from apswutils.db import NotFoundError
import math

In [ ]:
#| export
__all__ = ['AdminPanel', 'ModelAdmin']

In [ ]:
#| hide
from nbdev.showdoc import *

## Quick Start

Launch Kit's admin module provides a simple way to add admin interfaces to your FastHTML applications:

```python
from launch_kit.admin import AdminPanel, ModelAdmin
from fasthtml.common import *
from dataclasses import dataclass

# 1. Define your model as a dataclass
@dataclass
class Product:
    name: str
    price: float
    category: str
    is_active: bool = True
    id: Optional[int] = None

# 2. Create database and table
db = Database('app.db')
products = db.create(Product, pk='id')

# 3. Configure admin (optional)
@dataclass
class ProductAdmin(ModelAdmin):
    list_display = ['name', 'price', 'category', 'is_active']
    list_filter = ['category', 'is_active']
    search_fields = ['name']

# 4. Setup admin panel
app, rt = fast_app()
admin = AdminPanel(app, db)
admin.register(Product, products, ProductAdmin())

# That's it! Admin routes are automatically created
```

## Overview

This module provides comprehensive admin panel functionality:

### Core Components

| Component | Purpose | Key Features |
|-----------|---------|-------------|
| `AdminPanel` | Main admin interface | Model registration, route setup, auth integration |
| `ModelAdmin` | Model configuration | List display, filtering, search, ordering |

### Features

- **MiniDataAPI Native**: Built on FastHTML's database patterns
- **Dataclass-First**: Works with Python dataclasses, not raw schemas
- **MonsterUI Components**: Beautiful UI out of the box
- **HTMX Integration**: Dynamic updates without page reloads
- **Flexible & Transparent**: Every component can be customized

## Model Configuration

The `ModelAdmin` class configures how models appear and behave in the admin:

In [ ]:
#| export
@dataclass
class ModelAdmin:
    """Configuration for how a model appears in admin.
    
    All fields are optional with sensible defaults.
    """
    list_display: Optional[List[str]] = None        # Columns to show in list view
    list_filter: Optional[List[str]] = None         # Fields to filter by
    search_fields: Optional[List[str]] = None       # Fields to search
    ordering: Optional[List[str]] = None            # Default ordering (e.g., ['-created_at'])
    fields: Optional[List[str]] = None              # Fields to show in forms
    readonly_fields: Optional[List[str]] = None     # Fields that can't be edited
    actions: Optional[List[Callable]] = None        # Bulk actions
    per_page: int = 25                              # Items per page
    
    def __post_init__(self):
        """Initialize with empty lists if None."""
        self.list_display = self.list_display or []
        self.list_filter = self.list_filter or []
        self.search_fields = self.search_fields or []
        self.ordering = self.ordering or []
        self.fields = self.fields or []
        self.readonly_fields = self.readonly_fields or []
        self.actions = self.actions or []

In [ ]:
show_doc(ModelAdmin)

---

[source](https://github.com/LotsOfOrg/launch-kit/blob/main/launch_kit/admin.py#L24){target="_blank" style="float:right; font-size:smaller"}

### ModelAdmin

>      ModelAdmin (list_display:Optional[List[str]]=None,
>                  list_filter:Optional[List[str]]=None,
>                  search_fields:Optional[List[str]]=None,
>                  ordering:Optional[List[str]]=None,
>                  fields:Optional[List[str]]=None,
>                  readonly_fields:Optional[List[str]]=None,
>                  actions:Optional[List[Callable]]=None, per_page:int=25)

*Configuration for how a model appears in admin.

All fields are optional with sensible defaults.*

## AdminPanel Class

The main class that creates admin interfaces for your models:

In [ ]:
#| export
class AdminPanel:
    """Create admin panel for FastHTML app using MiniDataAPI.
    
    Provides automatic CRUD interfaces for registered models with
    customizable list views, forms, and actions.
    """
    
    def __init__(self, 
                 app,                              # FastHTML app instance
                 db: Database,                     # FastliteDB instance
                 path_prefix: str = "/admin",      # URL prefix for admin routes
                 auth_check: Optional[Callable] = None,  # Function(req, sess) -> bool
                 theme: Theme = Theme.blue         # MonsterUI theme
                 ):
        """Initialize admin panel.
        
        Args:
            app: FastHTML app instance
            db: Database instance (fastlite)
            path_prefix: URL prefix for admin routes
            auth_check: Optional auth check function
            theme: MonsterUI theme
        """
        self.app = app
        self.db = db
        self.path_prefix = path_prefix.rstrip('/')
        self.auth_check = auth_check or (lambda r, s: require_role('admin', r, s))
        self.theme = theme
        self.registered_models = {}
        
        # Setup admin routes
        self._setup_admin_routes()
    
    def register(self, 
                 model_class: Type,                # The dataclass model
                 table,                            # MiniDataAPI table (from db.create)
                 config: Optional[ModelAdmin] = None  # Optional configuration
                 ) -> None:
        """Register a dataclass model with admin.
        
        Args:
            model_class: The dataclass model
            table: MiniDataAPI table (from db.create)
            config: Optional ModelAdmin configuration
        """
        model_name = model_class.__name__.lower()
        self.registered_models[model_name] = {
            'class': model_class,
            'table': table,
            'config': config or ModelAdmin(),
            'name': model_class.__name__
        }
        self._setup_model_routes(model_name)

In [ ]:
show_doc(AdminPanel)

---

[source](https://github.com/LotsOfOrg/launch-kit/blob/main/launch_kit/admin.py#L49){target="_blank" style="float:right; font-size:smaller"}

### AdminPanel

>      AdminPanel (app, db:apswutils.db.Database, path_prefix:str='/admin',
>                  auth_check:Optional[Callable]=None,
>                  theme:monsterui.core.Theme=<Theme.blue: 'blue'>)

*Create admin panel for FastHTML app using MiniDataAPI.

Provides automatic CRUD interfaces for registered models with
customizable list views, forms, and actions.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| app |  |  | FastHTML app instance |
| db | Database |  | FastliteDB instance |
| path_prefix | str | /admin | URL prefix for admin routes |
| auth_check | Optional | None | Function(req, sess) -> bool |
| theme | Theme | Theme.blue | MonsterUI theme |

In [ ]:
#| export
def _setup_model_routes(self, model_name: str):
    """Setup CRUD routes for a model."""
    model_info = self.registered_models[model_name]
    
    # List view
    @self.app.route(f"{self.path_prefix}/{model_name}/")
    def list_view(req, sess):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        return self._render_list_view(model_name, req)
    
    # Create view (GET)
    @self.app.route(f"{self.path_prefix}/{model_name}/new")
    def create_form(req, sess):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        return self._render_form(model_name, None)
    
    # Create handler (POST)
    @self.app.route(f"{self.path_prefix}/{model_name}/new", methods=['POST'])
    async def create_handler(req, sess):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        form_data = await req.form()
        result, error = self._create_item(model_name, dict(form_data))
        
        if error:
            return self._render_form(model_name, None, error)
        
        # Redirect to list with success message
        return RedirectResponse(
            f"{self.path_prefix}/{model_name}/",
            status_code=303
        )
    
    # Edit view (GET)
    @self.app.route(f"{self.path_prefix}/{model_name}/<int:item_id>/edit")
    def edit_form(req, sess, item_id: int):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        table = model_info['table']
        try:
            item = table[item_id]
            # Convert to dict if needed
            if hasattr(item, '__dict__'):
                item = vars(item)
            return self._render_form(model_name, item)
        except NotFoundError:
            return HTMLResponse("Item not found", status_code=404)
    
    # Edit handler (POST)
    @self.app.route(f"{self.path_prefix}/{model_name}/<int:item_id>/edit", methods=['POST'])
    async def edit_handler(req, sess, item_id: int):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        form_data = await req.form()
        result, error = self._update_item(model_name, item_id, dict(form_data))
        
        if error:
            table = model_info['table']
            item = table[item_id]
            if hasattr(item, '__dict__'):
                item = vars(item)
            return self._render_form(model_name, item, error)
        
        return RedirectResponse(
            f"{self.path_prefix}/{model_name}/",
            status_code=303
        )
    
    # Delete handler (POST)
    @self.app.route(f"{self.path_prefix}/{model_name}/<int:item_id>/delete", methods=['POST'])
    def delete_handler(req, sess, item_id: int):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        if self._delete_item(model_name, item_id):
            # Return success for HTMX
            if req.headers.get('HX-Request'):
                return ""
        
        return RedirectResponse(
            f"{self.path_prefix}/{model_name}/",
            status_code=303
        )

# Bind method to class
AdminPanel._setup_model_routes = _setup_model_routes

In [ ]:
#| export
def _setup_admin_routes(self):
    """Setup main admin routes."""
    
    @self.app.route(self.path_prefix)
    def admin_dashboard(req, sess):
        # Check auth
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        # Create dashboard
        models_list = []
        for model_name, model_info in self.registered_models.items():
            count = len(model_info['table']())
            models_list.append(
                Card(
                    H3(model_info['name']),
                    P(f"{count} items", cls="text-muted-foreground"),
                    A("View →", href=f"{self.path_prefix}/{model_name}/", 
                      cls="text-primary hover:underline")
                )
            )
        
        return Container(
            H1("Admin Dashboard", cls="text-3xl font-bold mb-6"),
            Grid(*models_list, cols=3) if models_list else P("No models registered"),
            cls="py-8"
        )

# Bind method to class
AdminPanel._setup_admin_routes = _setup_admin_routes

### Model Routes Setup

In [ ]:
#| export
def _setup_model_routes(self, model_name: str):
    """Setup CRUD routes for a model."""
    model_info = self.registered_models[model_name]
    
    # List view
    @self.app.route(f"{self.path_prefix}/{model_name}/")
    def list_view(req, sess):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        return self._render_list_view(model_name, req)
    
    # Create view (GET)
    @self.app.route(f"{self.path_prefix}/{model_name}/new")
    def create_form(req, sess):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        return self._render_form(model_name, None)
    
    # Create handler (POST)
    @self.app.route(f"{self.path_prefix}/{model_name}/new", methods=['POST'])
    async def create_handler(req, sess):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        form_data = await req.form()
        result, error = self._create_item(model_name, dict(form_data))
        
        if error:
            return self._render_form(model_name, None, error)
        
        # Redirect to list with success message
        return RedirectResponse(
            f"{self.path_prefix}/{model_name}/",
            status_code=303
        )
    
    # Edit view (GET)
    @self.app.route(f"{self.path_prefix}/{model_name}/<int:item_id>/edit")
    def edit_form(req, sess, item_id: int):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        table = model_info['table']
        try:
            item = table[item_id]
            return self._render_form(model_name, item)
        except NotFoundError:
            return HTMLResponse("Item not found", status_code=404)
    
    # Edit handler (POST)
    @self.app.route(f"{self.path_prefix}/{model_name}/<int:item_id>/edit", methods=['POST'])
    async def edit_handler(req, sess, item_id: int):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        form_data = await req.form()
        result, error = self._update_item(model_name, item_id, dict(form_data))
        
        if error:
            table = model_info['table']
            item = table[item_id]
            return self._render_form(model_name, item, error)
        
        return RedirectResponse(
            f"{self.path_prefix}/{model_name}/",
            status_code=303
        )
    
    # Delete handler (POST)
    @self.app.route(f"{self.path_prefix}/{model_name}/<int:item_id>/delete", methods=['POST'])
    def delete_handler(req, sess, item_id: int):
        if not self.auth_check(req, sess):
            return RedirectResponse('/login', status_code=303)
        
        if self._delete_item(model_name, item_id):
            # Return success for HTMX
            if req.headers.get('HX-Request'):
                return ""
        
        return RedirectResponse(
            f"{self.path_prefix}/{model_name}/",
            status_code=303
        )

# Bind method to class
AdminPanel._setup_model_routes = _setup_model_routes

In [ ]:
#| export
def _build_list_page(self, model_name: str, items: List[Dict], 
                     page: int, total_pages: int, 
                     search: str, filters: Dict) -> FT:
    """Build the list page with table and controls."""
    model_info = self.registered_models[model_name]
    config = model_info['config']
    
    # Determine columns to display
    if config.list_display:
        columns = config.list_display
    else:
        # Use all fields except id
        columns = [f.name for f in fields(model_info['class']) if f.name != 'id']
    
    # Build table headers
    headers = [Th(col.replace('_', ' ').title()) for col in columns]
    headers.append(Th("Actions", cls="text-right"))
    
    # Build table rows
    rows = []
    for item in items:
        # Handle both dict and dataclass instances
        if hasattr(item, '__dict__'):
            item_dict = vars(item)
        else:
            item_dict = item
            
        row_cells = [Td(str(item_dict.get(col, ''))) for col in columns]
        
        # Add action buttons
        actions = Td(
            Div(
                A("Edit", href=f"{self.path_prefix}/{model_name}/{item_dict['id']}/edit",
                  cls="text-primary hover:underline mr-3"),
                Button("Delete", 
                       hx_post=f"{self.path_prefix}/{model_name}/{item_dict['id']}/delete",
                       hx_confirm="Are you sure?",
                       hx_target="closest tr",
                       hx_swap="outerHTML",
                       cls="text-destructive hover:underline"),
                cls="text-right"
            )
        )
        row_cells.append(actions)
        rows.append(Tr(*row_cells))
    
    # Build search form
    search_form = Form(
        Input(name="search", value=search, placeholder="Search...", 
              cls="mr-2"),
        Button("Search", type="submit"),
        method="get",
        cls="mb-4"
    )
    
    # Build pagination
    pagination = self._build_pagination(model_name, page, total_pages, search, filters)
    
    # Build page
    return Container(
        Div(
            H1(model_info['name'] + " List", cls="text-3xl font-bold"),
            A("+ New " + model_info['name'], 
              href=f"{self.path_prefix}/{model_name}/new",
              cls="btn btn-primary"),
            cls="flex justify-between items-center mb-6"
        ),
        search_form,
        Card(
            Table(
                Thead(Tr(*headers)),
                Tbody(*rows) if rows else Tbody(Tr(Td("No items found", colspan=len(headers)))),
                cls="w-full"
            )
        ),
        pagination,
        cls="py-8"
    )

# Bind method to class
AdminPanel._build_list_page = _build_list_page

In [ ]:
#| export
def _render_list_view(self, model_name: str, req) -> FT:
    """Render list view with pagination and filtering."""
    model_info = self.registered_models[model_name]
    table = model_info['table']
    config = model_info['config']
    
    # Get query parameters
    page = int(req.query_params.get('page', 1))
    search = req.query_params.get('search', '')
    sort_by = req.query_params.get('sort', config.ordering[0] if config.ordering else 'id')
    filters = {k: v for k, v in req.query_params.items() 
              if k not in ['page', 'search', 'sort'] and v}
    
    # Build query using MiniDataAPI
    query_args = []
    where_clauses = []
    
    # Search
    if search and config.search_fields:
        search_conditions = ' OR '.join([f"{field} LIKE ?" for field in config.search_fields])
        where_clauses.append(f"({search_conditions})")
        query_args.extend([f"%{search}%" for _ in config.search_fields])
    
    # Filters
    for field, value in filters.items():
        if field in config.list_filter:
            where_clauses.append(f"{field} = ?")
            query_args.append(value)
    
    # Execute query
    where = ' AND '.join(where_clauses) if where_clauses else None
    
    # Get total count
    total_items = len(table(where=where, where_args=query_args))
    total_pages = math.ceil(total_items / config.per_page)
    
    # Get paginated items
    offset = (page - 1) * config.per_page
    items = table(
        where=where,
        where_args=query_args,
        order_by=sort_by,
        limit=config.per_page,
        offset=offset
    )
    
    # Build table
    return self._build_list_page(model_name, items, page, total_pages, search, filters)

# Bind method to class
AdminPanel._render_list_view = _render_list_view

In [ ]:
#| export
def _build_pagination(self, model_name: str, page: int, total_pages: int,
                      search: str, filters: Dict) -> FT:
    """Build pagination controls."""
    if total_pages <= 1:
        return Div()  # No pagination needed
    
    # Build query string for pagination links
    query_parts = []
    if search:
        query_parts.append(f"search={search}")
    for k, v in filters.items():
        query_parts.append(f"{k}={v}")
    
    base_query = "&".join(query_parts)
    base_url = f"{self.path_prefix}/{model_name}/"
    
    # Build pagination links
    links = []
    
    # Previous link
    if page > 1:
        prev_url = f"{base_url}?page={page-1}&{base_query}" if base_query else f"{base_url}?page={page-1}"
        links.append(A("← Previous", href=prev_url, cls="btn btn-outline mr-2"))
    
    # Page numbers
    for p in range(max(1, page-2), min(total_pages+1, page+3)):
        if p == page:
            links.append(Span(str(p), cls="btn btn-primary mr-2"))
        else:
            page_url = f"{base_url}?page={p}&{base_query}" if base_query else f"{base_url}?page={p}"
            links.append(A(str(p), href=page_url, cls="btn btn-outline mr-2"))
    
    # Next link
    if page < total_pages:
        next_url = f"{base_url}?page={page+1}&{base_query}" if base_query else f"{base_url}?page={page+1}"
        links.append(A("Next →", href=next_url, cls="btn btn-outline"))
    
    return Div(*links, cls="mt-4 flex items-center")

# Bind method to class
AdminPanel._build_pagination = _build_pagination

## Form Generation

Generate forms from dataclass fields with appropriate input types:

In [ ]:
#| export
def _render_form(self, model_name: str, item: Optional[Dict] = None, 
                 error: Optional[str] = None) -> FT:
    """Render create/edit form."""
    model_info = self.registered_models[model_name]
    model_class = model_info['class']
    config = model_info['config']
    
    # Determine form action and title
    if item:
        action = f"{self.path_prefix}/{model_name}/{item['id']}/edit"
        title = f"Edit {model_info['name']}"
    else:
        action = f"{self.path_prefix}/{model_name}/new"
        title = f"New {model_info['name']}"
    
    # Generate form fields
    form_fields = self._generate_form_fields(model_class, config, item)
    
    # Build form
    form = Form(
        Alert(error, variant="destructive", cls="mb-4") if error else None,
        *form_fields,
        Div(
            Button("Save", type="submit", cls="btn btn-primary mr-2"),
            A("Cancel", href=f"{self.path_prefix}/{model_name}/", 
              cls="btn btn-outline"),
            cls="mt-4"
        ),
        method="post",
        action=action,
        cls="space-y-4"
    )
    
    return Container(
        H1(title, cls="text-3xl font-bold mb-6"),
        Card(form, cls="max-w-2xl"),
        cls="py-8"
    )

# Bind method to class
AdminPanel._render_form = _render_form

In [ ]:
#| export
def _generate_form_fields(self, model_class: Type, config: ModelAdmin, 
                          item: Optional[Dict] = None) -> List[FT]:
    """Generate form fields from dataclass."""
    form_fields = []
    
    # Get fields to display
    if config.fields:
        field_names = config.fields
    else:
        # All fields except id
        field_names = [f.name for f in fields(model_class) if f.name != 'id']
    
    # Generate input for each field
    for field_name in field_names:
        # Skip readonly fields in form
        if field_name in config.readonly_fields:
            continue
        
        # Get field info
        field_info = next((f for f in fields(model_class) if f.name == field_name), None)
        if not field_info:
            continue
        
        # Get current value
        if item:
            value = item.get(field_name, field_info.default)
        else:
            value = field_info.default if field_info.default != field_info.default_factory else ''
        
        # Generate appropriate input
        input_component = self._get_input_for_field(field_name, field_info, value)
        form_fields.append(input_component)
    
    return form_fields

# Bind method to class
AdminPanel._generate_form_fields = _generate_form_fields

In [ ]:
#| export
def _get_input_for_field(self, field_name: str, field_info, value: Any) -> FT:
    """Get appropriate input component for field type."""
    # Convert field name to label
    label = field_name.replace('_', ' ').title()
    
    # Handle different types
    field_type = field_info.type
    
    # Handle Optional types
    if hasattr(field_type, '__origin__'):
        if field_type.__origin__ is Union:
            # Get the non-None type from Optional[X]
            args = field_type.__args__
            field_type = next((t for t in args if t != type(None)), str)
    
    # Boolean -> Checkbox
    if field_type is bool:
        return Div(
            Label(
                Input(type="checkbox", name=field_name, value="true",
                      checked=bool(value)),
                label,
                cls="flex items-center space-x-2"
            )
        )
    
    # Numbers
    elif field_type in (int, float):
        input_type = "number"
        step = "0.01" if field_type is float else "1"
        return LabelInput(label, name=field_name, type=input_type, 
                         value=str(value) if value is not None else '',
                         step=step)
    
    # Date/DateTime
    elif field_type is datetime:
        # Format datetime for input
        if isinstance(value, datetime):
            value = value.strftime('%Y-%m-%dT%H:%M')
        return LabelInput(label, name=field_name, type="datetime-local", 
                         value=value or '')
    
    elif field_type is date:
        # Format date for input
        if isinstance(value, date):
            value = value.strftime('%Y-%m-%d')
        return LabelInput(label, name=field_name, type="date", 
                         value=value or '')
    
    # Text (default)
    else:
        # Check if it's a long text field
        if 'description' in field_name or 'content' in field_name or 'body' in field_name:
            return Div(
                Label(label, htmlFor=field_name),
                Textarea(value or '', name=field_name, id=field_name, 
                        rows=4, cls="textarea")
            )
        else:
            return LabelInput(label, name=field_name, type="text", 
                            value=value or '')

# Bind method to class
AdminPanel._get_input_for_field = _get_input_for_field

## CRUD Operations

Create, update, and delete operations following auth module patterns:

In [ ]:
#| export
def _create_item(self, model_name: str, form_data: Dict) -> tuple[Any, Optional[str]]:
    """Create new item using MiniDataAPI.
    
    Returns:
        Tuple of (created_item, error_message)
    """
    model_info = self.registered_models[model_name]
    model_class = model_info['class']
    table = model_info['table']
    
    try:
        # Clean and validate form data
        cleaned_data = self._clean_form_data(model_class, form_data)
        
        # Create dataclass instance
        instance = model_class(**cleaned_data)
        
        # Insert using MiniDataAPI
        result = table.insert(instance)
        return result, None
        
    except Exception as e:
        return None, str(e)

# Bind method to class
AdminPanel._create_item = _create_item

In [ ]:
#| export
def _delete_item(self, model_name: str, item_id: int) -> bool:
    """Delete item using MiniDataAPI.
    
    Returns:
        True if deleted, False if not found
    """
    model_info = self.registered_models[model_name]
    table = model_info['table']
    
    try:
        # Check if exists
        item = table[item_id]
        if item:
            table.delete(item_id)
            return True
    except NotFoundError:
        pass
    
    return False

# Bind method to class
AdminPanel._delete_item = _delete_item

In [ ]:
#| export
def _clean_form_data(self, model_class: Type, form_data: Dict) -> Dict:
    """Clean and convert form data to appropriate types."""
    cleaned = {}
    
    for field in fields(model_class):
        if field.name == 'id':  # Skip primary key
            continue
            
        value = form_data.get(field.name)
        
        # Handle empty strings
        if value == '':
            # Use default or None for optional fields
            if field.default != field.default_factory:
                value = field.default
            else:
                value = None
        
        # Type conversion
        if value is not None and value != '':
            field_type = field.type
            
            # Handle Optional types
            if hasattr(field_type, '__origin__') and field_type.__origin__ is Union:
                args = field_type.__args__
                field_type = next((t for t in args if t != type(None)), str)
            
            # Convert based on type
            if field_type is bool:
                value = value == 'true' or value == 'on'
            elif field_type is int:
                value = int(value)
            elif field_type is float:
                value = float(value)
            elif field_type is datetime:
                value = datetime.fromisoformat(value)
            elif field_type is date:
                value = datetime.strptime(value, '%Y-%m-%d').date()
        
        # Don't include None values for fields with defaults
        if value is not None or field.default == field.default_factory:
            cleaned[field.name] = value
    
    return cleaned

# Bind method to class
AdminPanel._clean_form_data = _clean_form_data

In [ ]:
#| export
def _update_item(self, model_name: str, item_id: int, form_data: Dict) -> tuple[Any, Optional[str]]:
    """Update item following auth module's patterns.
    
    Returns:
        Tuple of (updated_item, error_message)
    """
    model_info = self.registered_models[model_name]
    model_class = model_info['class']
    table = model_info['table']
    
    try:
        # Get existing item
        item = table[item_id]
        if not item:
            return None, "Item not found"
        
        # Convert to dict if needed
        if hasattr(item, '__dict__'):
            item_dict = vars(item)
        else:
            item_dict = item
            
        # Clean form data
        cleaned_data = self._clean_form_data(model_class, form_data)
        
        # Update item
        updated_item = dict(item_dict)
        updated_item.update(cleaned_data)
        
        # Update timestamp if field exists
        if 'updated_at' in model_class.__dataclass_fields__:
            updated_item['updated_at'] = datetime.utcnow()
        
        # Save using MiniDataAPI
        result = table.update(updated_item)
        return result, None
        
    except NotFoundError:
        return None, "Item not found"
    except Exception as e:
        return None, str(e)

# Bind method to class
AdminPanel._update_item = _update_item

In [ ]:
#| eval: false
from launch_kit.admin import AdminPanel, ModelAdmin
from launch_kit.auth import init_auth_tables
from fasthtml.common import *
from dataclasses import dataclass
from typing import Optional

# Initialize database
db = Database('demo.db')

# Define a model
@dataclass
class Product:
    name: str
    price: float
    category: str
    description: str = ""
    is_active: bool = True
    id: Optional[int] = None

# Create table
products = db.create(Product, pk='id')

# Setup app and admin
app, rt = fast_app()
admin = AdminPanel(app, db)
admin.register(Product, products)

# That's it! Visit /admin to see your admin panel

### Custom Configuration

In [ ]:
#| eval: false
# Configure how the model appears in admin
@dataclass
class ProductAdmin(ModelAdmin):
    list_display = ['name', 'price', 'category', 'is_active']
    list_filter = ['category', 'is_active']
    search_fields = ['name', 'description']
    ordering = ['-price']  # Sort by price descending
    per_page = 50

# Register with custom config
admin.register(Product, products, ProductAdmin())

### Complete Example with Auth

In [ ]:
#| eval: false
from launch_kit.admin import AdminPanel, ModelAdmin
from launch_kit.auth import init_auth_tables, User
from launch_kit.permissions import role_required
from fasthtml.common import *

# Initialize database with auth tables
db = init_auth_tables('app.db')

# Create app
app, rt = fast_app()

# Setup admin with auth check
admin = AdminPanel(
    app, 
    db,
    auth_check=lambda r, s: require_role('admin', r, s)
)

# Configure User admin
@dataclass
class UserAdmin(ModelAdmin):
    list_display = ['username', 'email', 'role', 'is_active', 'created_at']
    list_filter = ['role', 'is_active']
    search_fields = ['username', 'email']
    ordering = ['-created_at']
    readonly_fields = ['password_hash', 'created_at', 'updated_at']

# Register User model
# Note: init_auth_tables creates a table named 'user' (singular)
admin.register(User, db['user'], UserAdmin())

In [ ]:
#| hide
# Test basic functionality
import tempfile
import os

print("Testing AdminPanel...")

# Create test database
with tempfile.TemporaryDirectory() as tmpdir:
    test_db_path = os.path.join(tmpdir, 'test_admin.db')
    db = Database(test_db_path)
    
    # Define test model
    @dataclass
    class TestModel:
        name: str
        value: int
        is_active: bool = True
        id: Optional[int] = None
    
    # Create table
    test_table = db.create(TestModel, pk='id')
    
    # Test ModelAdmin defaults
    config = ModelAdmin()
    assert config.per_page == 25
    assert config.list_display == []
    print("✓ ModelAdmin defaults working")
    
    # Create test app
    from starlette.testclient import TestClient
    test_app, test_rt = fast_app()
    
    # Create admin panel
    admin = AdminPanel(
        test_app, 
        db,
        auth_check=lambda r, s: True  # Always authenticated for tests
    )
    
    # Register model
    admin.register(TestModel, test_table)
    
    # Verify registration
    assert 'testmodel' in admin.registered_models
    assert admin.registered_models['testmodel']['class'] == TestModel
    print("✓ Model registration working")
    
    # Test CRUD operations
    # Create
    result, error = admin._create_item('testmodel', {
        'name': 'Test Item',
        'value': '42',
        'is_active': 'true'
    })
    assert error is None
    # MiniDataAPI returns instances, convert to dict for testing
    if hasattr(result, '__dict__'):
        result_dict = vars(result)
    else:
        result_dict = result
    assert result_dict['name'] == 'Test Item'
    assert result_dict['value'] == 42
    assert result_dict['is_active'] == True
    item_id = result_dict['id']
    print("✓ Create operation working")
    
    # Update
    result, error = admin._update_item('testmodel', item_id, {
        'name': 'Updated Item',
        'value': '100',
        'is_active': ''
    })
    assert error is None
    if hasattr(result, '__dict__'):
        result_dict = vars(result)
    else:
        result_dict = result
    assert result_dict['name'] == 'Updated Item'
    assert result_dict['value'] == 100
    print("✓ Update operation working")
    
    # Delete
    deleted = admin._delete_item('testmodel', item_id)
    assert deleted == True
    print("✓ Delete operation working")
    
    # Test form data cleaning
    cleaned = admin._clean_form_data(TestModel, {
        'name': 'Test',
        'value': '123',
        'is_active': 'on',
        'extra_field': 'ignored'
    })
    assert cleaned == {'name': 'Test', 'value': 123, 'is_active': True}
    print("✓ Form data cleaning working")
    
print("\n✅ All AdminPanel tests passed!")

Testing AdminPanel...
✓ ModelAdmin defaults working
✓ Model registration working
✓ Create operation working
✓ Update operation working
✓ Delete operation working
✓ Form data cleaning working

✅ All AdminPanel tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Interactive Demo

Try the admin panel with a complete example:

In [ ]:
#| eval: false
# Complete demo with JupyUvi
from fasthtml.jupyter import JupyUvi
from launch_kit.admin import AdminPanel, ModelAdmin
from fasthtml.common import *
from dataclasses import dataclass, field
from datetime import datetime
from typing import Optional

# Create demo database
demo_db = Database(':memory:')

# Define models
@dataclass
class Article:
    title: str
    content: str
    author: str
    category: str = "general"
    is_published: bool = False
    created_at: datetime = field(default_factory=datetime.utcnow)
    updated_at: datetime = field(default_factory=datetime.utcnow)
    id: Optional[int] = None

# Create tables
articles = demo_db.create(Article, pk='id')

# Insert sample data
for i in range(1, 11):
    articles.insert(Article(
        title=f"Article {i}",
        content=f"This is the content of article {i}. It contains important information.",
        author=f"Author {i % 3 + 1}",
        category=["tech", "science", "general"][i % 3],
        is_published=i % 2 == 0
    ))

# Configure admin
@dataclass
class ArticleAdmin(ModelAdmin):
    list_display = ['title', 'author', 'category', 'is_published', 'created_at']
    list_filter = ['category', 'is_published']
    search_fields = ['title', 'content', 'author']
    ordering = ['-created_at']
    per_page = 5

# Create app
demo_app, demo_rt = fast_app(hdrs=Theme.green.headers())

# Setup admin (no auth for demo)
admin = AdminPanel(
    demo_app, 
    demo_db,
    auth_check=lambda r, s: True  # Always allow access for demo
)

# Register models
admin.register(Article, articles, ArticleAdmin())

# Add home page
@demo_rt('/')
def home():
    return Container(
        H1("Demo App", cls="text-3xl font-bold mb-4"),
        P("This is a demo of the Launch Kit admin panel."),
        A("Go to Admin →", href="/admin", cls="btn btn-primary mt-4"),
        cls="py-8"
    )

# Run the demo
print("Starting demo server...")
print("\nVisit these URLs:")
print("- http://localhost:8000/ - Home page")
print("- http://localhost:8000/admin - Admin dashboard")
print("- http://localhost:8000/admin/article/ - Article list")
print("\nTry searching, filtering, and editing articles!")

server = JupyUvi(demo_app)

<string>:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
<string>:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).


Starting demo server...

Visit these URLs:
- http://localhost:8000/ - Home page
- http://localhost:8000/admin - Admin dashboard
- http://localhost:8000/admin/article/ - Article list

Try searching, filtering, and editing articles!


In [ ]:
#| eval: false
# Stop the server when done
server.stop()

## Summary

Launch Kit's admin module provides:

- **MiniDataAPI Integration** - Built on FastHTML's database patterns
- **Dataclass-First** - Works with Python dataclasses, not schemas
- **Zero Configuration** - Works out of the box with sensible defaults
- **Fully Customizable** - Every aspect can be configured
- **MonsterUI Components** - Beautiful, responsive UI
- **HTMX Powered** - Dynamic updates without page reloads
- **Production Ready** - Authentication, pagination, search, and filtering

The module follows Launch Kit's philosophy of being simple, transparent, and flexible while providing all the features needed for production admin interfaces.